## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import models, datasets, transforms
from sklearn.model_selection import StratifiedKFold
import numpy as np
from utils import train_model, evaluate_model
import random

## Set seed for reproducibility

In [2]:
def set_seed(seed):
    """
    Set the seed for reproducibility.

    Args:
        seed (int): Seed value to set for random number generation.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed to be 42
set_seed(42)

## EfficientNet-B2

In [3]:
# Use CUDA if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the EfficientNet model to use
def create_efficientnet_model(num_classes):
    # Load the EfficientNet-B2 model
    model = models.efficientnet_b3(pretrained=False)
    # Replace the classifier layer
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model.to(device)

## Helper Functions

In [4]:
import itertools
import torch
import numpy as np

def load_dataset(dataset_folder):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.436, 0.385, 0.344], std=[0.296, 0.269, 0.261])
    ])
    
    dataset = datasets.ImageFolder(root=dataset_folder, transform=transform)
    return dataset
    
def grid_search_tuning(dataset, num_classes, learning_rate, batch_sizes, num_epochs=5):
    best_accuracy = 0
    best_f1 = 0
    best_params = {}

    for lr, bs in itertools.product(learning_rates, batch_sizes):
        print(f"Testing LR: {lr}, BS: {bs}")

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        targets = np.array([dataset.targets[i] for i in range(len(dataset))])
        fold_accuracies = []
        fold_f1_scores = []

        for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(dataset)), targets)):
            print(f'Fold {fold + 1}')
            
            train_sampler = Subset(dataset, train_idx)
            val_sampler = Subset(dataset, val_idx)
            
            train_dataloader = DataLoader(train_sampler, batch_size=bs, shuffle=True)
            val_dataloader = DataLoader(val_sampler, batch_size=bs, shuffle=False)
            
            model = create_efficientnet_model(num_classes)
            trained_model = train_model(model, train_dataloader, val_dataloader, num_epochs=num_epochs, lr=lr)
            
            fold_accuracy, fold_f1 = evaluate_model(trained_model, val_dataloader)
            fold_accuracies.append(fold_accuracy)
            fold_f1_scores.append(fold_f1)
            print(f'Fold {fold + 1} Accuracy: {fold_accuracy:.4f}, F1 Score: {fold_f1:.4f}')
        
        mean_accuracy = np.mean(fold_accuracies)
        mean_f1 = np.mean(fold_f1_scores)
        print(f'Mean Accuracy for LR: {lr}, BS: {bs}: {mean_accuracy:.4f}')
        print(f'Mean F1 Score for LR: {lr}, BS: {bs}: {mean_f1:.4f}')

        if mean_accuracy > best_accuracy:
            best_accuracy = mean_accuracy
            best_f1 = mean_f1
            best_params = {'learning_rate': lr, 'batch_size': bs}
    
    print(f'Best Parameters: {best_params}')
    print(f'Best Accuracy: {best_accuracy:.4f}')
    print(f'Best F1 Score: {best_f1:.4f}')

## Start the tuning

In [5]:
# Set up device and dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_folder = '../data/lfw'
dataset = load_dataset(dataset_folder)
num_classes = len(dataset.classes)

In [6]:
# Define ranges for hyperparameters
learning_rates = [0.01, 0.1]
batch_sizes = [32, 64, 16, 8]

# Perform grid search tuning
grid_search_tuning(dataset, num_classes, learning_rates, batch_sizes, num_epochs=50)

Testing LR: 0.01, BS: 32
Fold 1
Epoch 1/50, Train Loss: 3.0650, Val Loss: 3.5697
Epoch 2/50, Train Loss: 2.5808, Val Loss: 2.3956
Epoch 3/50, Train Loss: 2.4287, Val Loss: 2.5252
Epoch 4/50, Train Loss: 2.4000, Val Loss: 2.2771
Epoch 5/50, Train Loss: 2.3635, Val Loss: 2.2756
Epoch 6/50, Train Loss: 2.3674, Val Loss: 2.3313
Epoch 7/50, Train Loss: 2.3513, Val Loss: 2.3393
Epoch 8/50, Train Loss: 2.3600, Val Loss: 2.2892
Epoch 9/50, Train Loss: 2.3377, Val Loss: 2.3650
Epoch 10/50, Train Loss: 2.3578, Val Loss: 2.3156
Early stopping triggered after 10 epochs
Training complete
Accuracy: 0.0582
F1 Score: 0.0208
Fold 1 Accuracy: 0.0582, F1 Score: 0.0208
Fold 2
Epoch 1/50, Train Loss: 2.9349, Val Loss: 3.7154
Epoch 2/50, Train Loss: 2.4969, Val Loss: 2.5378
Epoch 3/50, Train Loss: 2.3958, Val Loss: 2.2825
Epoch 4/50, Train Loss: 2.3690, Val Loss: 2.2046
Epoch 5/50, Train Loss: 2.3326, Val Loss: 2.1964
Epoch 6/50, Train Loss: 2.3074, Val Loss: 2.3216
Epoch 7/50, Train Loss: 2.3533, Val Loss:

## Results

```
Best Parameters: {'learning_rate': 0.01, 'batch_size': 4}

```